# Feature selection

## La feature selection, c'est quoi ? 

La feature selection, c'est le fait de choisir quelles variables explicatives on souhaite garder pour mettre en place notre modèle. On va chercher à limiter le nombre de colonnes en minimisant la perte d'informations provenant de la suppression des autres colonnes en éliminant les données redondantes ou inutiles pour notre étude. 

## Pourquoi faire cette selection ? 

La feature selection a plusieurs intérêts :
- simplifier le modèle pour le rendre plus facile à interpréter (lors de la mise en place du modèle et pour l'utilisateur)
- réduire le temps d'entrainement et le temps d'exécution du programme
- éviter la "curse of dimensionality" (fléau de la dimension):
    Quand le nombre de features augmente, le nombre de données nécessaires à la généralisation augmente exponentiellement.  
    Illustration de la curse of dimensionality : 
![Illustration de la curse of dimensionality](https://www.researchgate.net/profile/Xavier-Hadoux/publication/278786166/figure/fig9/AS:613951482101777@1523388848382/Illustration-of-the-Curse-of-dimensionality-showing-how-the-number-of-regions-of-a.png)
- améliorer la généralisation en réduisant l'overfitting 

## Quels sont les différents moyens de réaliser de la feature selection avec scikit ? 

Scikit met à disposition un module permettant de faire de la feature selection : ```sklearn.feature_selection```.
On va parcourir l'ensemble des outils mis à disposition. 

### Sélection des features ayant une variance suffisante
#### [VarianceThreshhold](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold)

Cet algorithme permet de retirer toutes les colonnes ayant une variance faible, inférieure à un seuil  ```threshhold``` qu'on peut fixer. Il ne s'applique que aux variables explicatives et peut donc être également utilisé en apprentissage non supervisé.  
L'intérêt est de se débarasser de colonnes ayant (quasiment) tout le temps la même valeur pour toutes les lignes et qui n'apportent donc pas d'indication suffisamment pertinente pour obtenir la valeur cible.  
Exemple d'un selecteur permettant de retirer les colonnes qui contiennent plus de 80% de la même valeur: 
```python
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
```

### Sélection univariée des features

On peut sélectionner les colonnes suivant des tests statistiques univariés qui utiliseront la dépendance entre nos colonnes explicatives et la variable cible. Celà permet de garder les colonnes les plus proches et liés à la variable cible. 

#### [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest)

Garde uniquement les ```K``` features ayant le plus haut score. 

#### [SelectPercentile](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile)

Garde uniquement un pourcentage donné des features ayant le plus haut score.

#### [SelectFpr](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFpr.html#sklearn.feature_selection.SelectFpr)

Garde les colonnes ayant une pvalue sous un seuil donné basé sur un test de taux de faux positifs (FPR).

#### [SelectFdr](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFdr.html#sklearn.feature_selection.SelectFdr)

Garde les colonnes ayant une pvalue sous un seuil donné basé sur le False Discovery Rate (FDR). Celà permet d'éliminer les colonnes ne respectant pas l'hypothèse nulle.

#### [SelectFwe](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFwe.html#sklearn.feature_selection.SelectFwe)

Garde les colonnes ayant une pvalue sous un seuil donné basé sur le Family-wise Error Rate (Fwe). Celà permet de contrôler si des groupes de colonnes respectent l'hypothèse nulle. C'est un contrôle plus strict que celui de SelectFdr. 

#### [GenericUnivariateSelect](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.GenericUnivariateSelect.html#sklearn.feature_selection.GenericUnivariateSelect)

Sélecteur unviarié dont la stratégie est configurable. On peut lui faire utiliser l'ensemble des stratégies ci-dessus. 

Ces sélecteurs demandent une fonction renvoyant un score et une pvalue en entrée. Les fonctions mises à disposition sont les suivantes:  
- Pour la régression [f_regression](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression), [mutual_info_regression](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html#sklearn.feature_selection.mutual_info_regression)
- Pour la classification 
[chi2](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2), [f_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif), [mutual_info_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)  
  
On peut également définir ses propres fonctions, tant qu'elles renvoient un score et une pvalue.
  
Les méthodes basées sur le f-test utilisent **le niveau de dépendance linéaire** entre les variables. Les autres permettent de se baser sur **n'importe quel niveau de dépendance** mais nécessitent davantage de données pour être efficaces.  
Attention à bien utiliser un score adapté au problème. 

### Sélection avec [SelectFromModel](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel)

SelectFromModel est un meta-transformeur qui peut être utilisé avec n'importe quel estimateur donnant l'importance des features via un attribut (par exemple coeff_ ou feature_importances_). On ne garde que les features qui auront une valeur supérieur à un ```threshhold``` donné. On peut utiliser la moyenne ou la mediane des coefficients pour déterminer ce ```threshhold``` (par exemple "mean/10"). On peut également utiliser le paramètre ```max_features``` qui limite le nombre de features à sélectionner.  
Pour les **modèles linéaires** , les estimateurs les plus efficaces avec SelectFromModel sont ceux basés sur l1 pour éliminer les colonnes dont les coefficients sont nuls. On retrouve donc [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso) pour la régression linéaire et [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression) ainsi que [LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC) pour la classification linéaire.  
On peut également utiliser l'ensemble des modèles basés sur les **arbres de décision** (et les forêts) avec SelectFromModel pour retirer les colonnes ne contenant pas d'informations utiles. 

### Elimination récursive des features

En prenant un estimateur donnant l'importance des features comme pour SelectFromModel, l'algorithme va s'appliquer de façon récursive. Il va chercher à obtenir un nombre de plus en plus petit de features en se basant sur l'importance de ces features à chaque itération et en se débarassant de celles qui ont l'importance la plus faible jusqu'à atteindre le nombre de features désiré.  
Il existe 2 algorithmes pour effectuer cette élimination récursive : [RFE](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE) qui est l'algorithme de base et [RFECV](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html#sklearn.feature_selection.RFECV) qui permet de faire une crossvalidation dans l'algorithme pour sélectionner automatiquement le nombre optimal de features à conserver.

### Sélection séquentielle des features

[SequentialFeatureSelector](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html#sklearn.feature_selection.SequentialFeatureSelector) est un algorithme de sélection itératif qui a 2 modes de fonctionnement :
- **Forward-SFS** qui commence avec aucune feature et qui va sélectionner la meilleur feature à ajouter à son ensemble de features déjà utilisées à chaque itération. 
- **Backward-SFS** qui commence avec toutes les features et qui va retirer à chaque itération la feature la moins utile.  

L'algorithme s'arrête lorsqu'il atteint le nombre de features fixé par ```n_features_to_select```. Le mode est choisi via la paramètre ```direction```.  
Attention : suivant le mode choisi, on a pas le même nombre d'itérations ni le même résultat. 

La différence avec les modèles précédents est que ce sélecteur n'a pas besoin que le modèle lui donne un attribut de ```coeff_``` ou de ```feature_importance_```. Le sélecteur va directement comparer les modèles pour chaque jeu de features, ce qui fait que cet algorithme est plutôt lent. Il va donc évaluer plusieurs modèles pour chaque itération contrairement aux autres (par exemple RFE) qui n'évalue qu'un seul modèle par itération.  

 ### Intégration à un pipeline
 
 Il est recommandé d'intégrer la feature selection à un pipeline pour l'appliquer systématiquement aux jeux de données sur lequel on appliquera par la suite le modèle.